In [1]:
# pycuda, numpy, scipy, matplotlib
import os

os.environ["CUDA_DEVICE"] = "3"
from pycuda.autoinit import context
import pycuda.driver as drv
from pycuda import cumath
from pycuda import gpuarray
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import e, epsilon_0

from pypic import PyPIC_GPU as PIC
from meshing import RectMesh2D
from poisson_solver import FFT_solver as FFT

import sys
sys.path.append('../')
sys.path.append('old/')
import FFT_OpenBoundary_SquareGrid as PIC_old
import geom_impact_poly as poly
import geom_impact_ellip as ell

%matplotlib inline

PyKLU not found


## define mesh, poisson solver and pic

In [2]:
nx, ny = 256, 256
dx = dy = 0.04
x_aper = nx*dx/2.
y_aper = ny*dy/2.
mesh = RectMesh2D(-x_aper, -y_aper, dx, dy, nx, ny, mathlib=cumath)
poissonsolver = FFT.GPUFFTPoissonSolver2D(mesh)
pic = PIC(mesh, poissonsolver, context=context)

## generate particles
make sure they're inside the mesh boundaries

In [3]:
N = 500*1024
np.random.seed(0)
mesh_center_x = mesh.x0 + 0.5*mesh.nx*mesh.dx
mesh_center_y = mesh.y0 + 0.5*mesh.ny*mesh.dy
sigma = 0.4
xx = np.random.normal(mesh_center_x, sigma, N)
yy = np.random.normal(mesh_center_y, sigma, N)
assert((xx > mesh.x0).all() and (xx < mesh.x0 + mesh.nx*mesh.dx).all())
assert((yy > mesh.y0).all() and (yy < mesh.y0 + mesh.ny*mesh.dy).all())
xx = gpuarray.to_gpu(xx)
yy = gpuarray.to_gpu(yy)

### convenience wrapper which runs the pic and stores all intermediate results

In [4]:
def test_solver(new_solver, xx, yy, charge=e, x_probe=None, y_probe=None, z_probe=None):
    '''
    Return rho, Phi, efx, efy, Ex, Ey
    '''
    if x_probe == None:
        x_probe = xx
    if y_probe == None:
        y_probe = yy
    mesh_charges = new_solver.particles_to_mesh(xx, yy, charge=charge)
    rho = mesh_charges/new_solver.mesh.volume_elem
    phi = new_solver.poisson_solve(rho)
    mesh_e_fields = new_solver.get_electric_fields(phi)
    efx = mesh_e_fields[0]
    efy = mesh_e_fields[1]
    mesh_fields_and_mp_coords = zip(list(mesh_e_fields), [x_probe, y_probe])
    fields = new_solver.field_to_particles(*mesh_fields_and_mp_coords)
    Ex = fields[0]
    Ey = fields[1]
    return rho, phi, efx, efy, Ex, Ey

## Run the pic and store the results

In [5]:
rho, phi, efx, efy, Ex, Ey = test_solver(pic, xx, yy)

ArgumentError: Python argument types in
    Memcpy2D.__call__(Memcpy2D)
did not match C++ signature:
    __call__(pycuda::memcpy_2d {lvalue}, pycuda::stream)
    __call__(pycuda::memcpy_2d {lvalue} self, bool aligned)

### Run the old solver for comparison

In [ ]:
print rho
pic_old = PIC_old.FFT_OpenBoundary_SquareGrid(x_aper=x_aper, y_aper=y_aper, Dh = mesh.dx, fftlib='pyfftw')
nel_part = np.zeros(len(xx.get()))+1
pic_old.scatter(xx.get(), yy.get(), nel_part, charge=e)
pic_old.solve()

### Analytical solution for a round beam in free space

In [ ]:
def _phin_round(x, y, sig_r):
    '''Return phi / Q for a round distribution with
    sigma_x == sigma_y == sigma_z == sig_r .
    '''
    r = np.sqrt(x**2 + y**2 )
    #return erf(r/(np.sqrt(2)*sig_r)) *(-1)*np.log(r)/ (2*np.pi*epsilon_0)
    return (-1)*np.log(r)/ (2*np.pi*epsilon_0)

from scipy.special import erf
extent = np.array([mesh.x0, mesh.x0+mesh.nx*mesh.dx,
                   mesh.y0, mesh.y0+mesh.ny*mesh.dy])
y, x = np.meshgrid(   np.linspace(extent[2], extent[3], mesh.ny),
                      np.linspace(extent[0], extent[1], mesh.nx),
                      indexing="ij")
xr = x - mesh_center_x - 0.5*mesh.dx
yr = y - mesh_center_y - 0.5*mesh.dy
sig_x = sigma
phi_analytic = _phin_round(xr, yr, sig_x) * N * e

## Plot the results

In [ ]:
# rho
plt.figure()
plt.imshow(rho.get(), origin='lower', interpolation='none')
plt.colorbar()
plt.title('rho')
plt.xlabel('x [cell]')
plt.ylabel('y [cell]')
plt.show()

# Phi 2d
f = plt.figure()
f.set_figheight(8)
f.set_figwidth(15)
im =plt.imshow(phi.get(), origin='lower', interpolation='none')
plt.title('Phi')
plt.xlabel('x [cell]')
plt.ylabel('y [cell]')
f.subplots_adjust(right=0.8)
cbar_ax = f.add_axes([0.85, 0.15, 0.05, 0.7])
f.colorbar(im, cax=cbar_ax)
plt.show()

# Phi compare with analytical results and a 1/r fit
def fn(r, a):
    '''Analytical solution fit'''
    return a*1./np.log(r)

xxx = np.arange(mesh.x0, mesh.x0+mesh.nx*mesh.dx, dx)
start_fit = mesh.nx/2 + 4
fit_xxx = xxx[start_fit:]
#popt_GPU, _ = curve_fit(fn, fit_xxx, phi.get()[mesh.ny//2, start_fit:])
f, axarr = plt.subplots(2, sharex=True)
f.set_figheight(10)
f.set_figwidth(10)
axarr[0].set_title('Phi along ny/2')
axarr[0].plot(xxx,phi.get()[mesh.ny//2,:],label = 'numerical solution')
axarr[0].plot(xxx, phi_analytic[mesh.ny//2, :], label= 'analytical solution')
#axarr[0].plot(fit_xxx, fn(fit_xxx, *popt_GPU), label='log fit')
axarr[0].set_ylabel('Phi')
axarr[0].set_xlabel('x')

axarr[0].legend()
axarr[1].set_title('Particle distribution')
axarr[1].hist(xx.get())
axarr[1].set_xlabel('x')
axarr[1].set_ylabel('# particles')

plt.show()

plt.figure()
plt.plot(xxx, phi.get()[mesh.ny//2,:], label='numerical')
plt.show()

plt.figure()
plt.plot(pic_old.phi.T[mesh.ny//2,:], label='numerical')
plt.show()

In [ ]:
mat = gpuarray.zeros((3,3), dtype=np.complex128)
itemsize = np.dtype(np.complex128).itemsize
target = gpuarray.zeros((5, 5), dtype=np.complex128) + 1
print type(mat)
print mat.shape
print type(target)
print target.shape
copy = drv.Memcpy2D()
copy.set_src_device(mat.gpudata)
copy.set_dst_device(target.gpudata)
copy.src_pitch = mat.shape[1] * itemsize
copy.dst_pitch =  target.shape[1] * itemsize
copy.width_in_bytes = 3*itemsize
copy.height = 1
print 'src pitch: ' + str(copy.src_pitch)
print 'dst pitch: ' + str(copy.dst_pitch)
print 'height: ' + str(copy.height)

print mat
print target
copy(aligned=True)
print target

In [ ]:

# rho
plt.figure()
plt.imshow(rho.get(), origin='lower', interpolation='none')
plt.colorbar()
plt.title('rho')
plt.xlabel('x [cell]')
plt.ylabel('y [cell]')
plt.show()

# Phi 2d
f = plt.figure()
f.set_figheight(8)
f.set_figwidth(15)
im =plt.imshow(phi.get(), origin='lower', interpolation='none')
plt.title('Phi')
plt.xlabel('x [cell]')
plt.ylabel('y [cell]')
f.subplots_adjust(right=0.8)
cbar_ax = f.add_axes([0.85, 0.15, 0.05, 0.7])
f.colorbar(im, cax=cbar_ax)
plt.show()

# Phi compare with analytical results and a 1/r fit
def fn(r, a):
    '''Analytical solution fit'''
    return a*1./np.log(r)

xxx = np.arange(mesh.x0, mesh.x0+mesh.nx*mesh.dx, dx)
start_fit = mesh.nx/2 + 4
fit_xxx = xxx[start_fit:]
#popt_GPU, _ = curve_fit(fn, fit_xxx, phi.get()[mesh.ny//2, start_fit:])
f, axarr = plt.subplots(2, sharex=True)
f.set_figheight(10)
f.set_figwidth(10)
axarr[0].set_title('Phi along ny/2')
axarr[0].plot(xxx,phi.get()[mesh.ny//2,:],label = 'numerical solution')
axarr[0].plot(xxx, phi_analytic[mesh.ny//2, :], label= 'analytical solution')
#axarr[0].plot(fit_xxx, fn(fit_xxx, *popt_GPU), label='log fit')
axarr[0].set_ylabel('Phi')
axarr[0].set_xlabel('x')

axarr[0].legend()
axarr[1].set_title('Particle distribution')
axarr[1].hist(xx.get())
axarr[1].set_xlabel('x')
axarr[1].set_ylabel('# particles')

plt.show()

plt.figure()
plt.plot(xxx, phi.get()[mesh.ny//2,:], label='numerical')
plt.show()

plt.figure()
plt.plot(pic_old.phi.T[mesh.ny//2,:], label='numerical')
plt.show()